# Solvating and equilibrating a ligand in a box of water

<details>
    <summary><small>▼ Click here for dependency installation instructions</small></summary>
    The simplest way to install dependencies is to use the Interchange examples environment. From the root of the cloned openff-interchange repository:
    
    conda env create --name interchange-examples --file devtools/conda-envs/examples_env.yaml 
    conda activate interchange-examples
    pip install -e .
    cd examples
    jupyter notebook ligand_in_water.ipynb
    
</details>

In [ ]:
import time

import mdtraj
import openmm
import openmm.app
from openff.toolkit import ForceField, Molecule, Topology
from openff.units import unit
from openff.units.openmm import from_openmm, to_openmm

from openff.interchange import Interchange

## Construct the topology

In this example we'll construct a topology consisting of one ligand in a cubic box of length 4 nanometers. For simplicity, we will use a prepared PDB file  (`solvated.pdb`) with the same number of waters, molecule and atom ordering, etc. We'll also use _mapped_ SMILES when creating `Molecule` objects to ensure the atom ordering matches. (Atom ordering is not strictly a part of SMILES and therefore liable to be changed with updates to RDKit.)

This can be extended or modified by i.e.
* Replacing this sample ligand with a different ligand of interest - substitute out the ligand SMILES
* Using a different number of water molecules - substitute out the `2100` used below
* Adding ions or co-solvents into the box - add more `Molecule` object as desired

For each of these modifications, a new PDB file would need to be generated.

In [ ]:
ligand = Molecule.from_mapped_smiles(
    "[H:7][C@:6]1([C:13](=[C:11]([C:9](=[O:10])[O:8]1)[O:12][H:19])[O:14][H:20])[C@:3]([H:4])([C:2]([H:16])([H:17])[O:1][H:15])[O:5][H:18]"
)
water = Molecule.from_mapped_smiles("[H:2][O:1][H:3]")

There are a few ways to convert the information in this trajectory to an Openff [`Topology`](https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.topology.Topology.html#openff.toolkit.topology.Topology) object. In this case, since we already know how many of which molecules we want, we'll use [`Topology.from_molecules`](https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.topology.Topology.html#openff.toolkit.topology.Topology.from_molecules), which takes a list of `Molecule` objects and assembles them into a `Topology`.

In [ ]:
topology = Topology.from_molecules([ligand, *2100 * [water]])

We'll also set the box vectors to match the prepared PDB file.

In [ ]:
topology.box_vectors = unit.Quantity(
    mdtraj.load("solvated.pdb").unitcell_lengths[0],
    unit.nanometer,
)

And finally, set the positions of each molecule according to the data in the MDTraj object:

The ["Sage"](https://openforcefield.org/community/news/general/sage2.0.0-release/) force field line (version 2.x.x) includes TIP3P  parameters for water, so we don't need to use multiple force fields to parametrize this topology as long as we're okay using TIP3P.

Note that the "Parsley" (version 1.x.x) line did *not* include TIP3P parameters, so loading in an extra force field was required.

In [ ]:
sage = ForceField("openff-2.0.0.offxml")

From here, we can create an ``Interchange`` object and promptly export it to an [``openmm.System``](http://docs.openmm.org/latest/api-python/generated/openmm.openmm.System.html#openmm.openmm.System):

Note that these two steps (creating an `Interchange` object and exporting it to an [``openmm.System``](http://docs.openmm.org/latest/api-python/generated/openmm.openmm.System.html#openmm.openmm.System)) can equivalently be done in one step via [``ForceField.create_openmm_system``](https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.typing.engines.smirnoff.forcefield.ForceField.html#openff.toolkit.typing.engines.smirnoff.forcefield.ForceField.create_openmm_system).

In [ ]:
interchange: Interchange = Interchange.from_smirnoff(
    force_field=sage, topology=topology
)
system: openmm.System = interchange.to_openmm(combine_nonbonded_forces=True)

Note that these two lines do essentially the same thing as calling `sage.create_openmm_system(topology)`, which can be used to the same result if only interested in using OpenMM. Here we explicitly store the intermediate `Interchange` object for later steps.

Finally, we need to set the positions according to the PDB file. There are plenty of ways to extract positions from a PDB file, here we'll use MDTraj.

In [ ]:
interchange.positions = unit.Quantity(
    mdtraj.load("solvated.pdb").xyz[0],
    unit.nanometer,
)

Now, we can prepare everything else that OpenMM needs to run and report a brief equilibration simulation:
* A barostat, since we want to use NPT dynamics to relax the box size toward equilibrium
* An integrator
* A [`Simulation`](http://docs.openmm.org/latest/api-python/generated/openmm.app.simulation.Simulation.html#openmm.app.simulation.Simulation) object, putting it together
* Reporters for the trajectory and simulation data

For convenience, let's wrap some boilerplate code into a function that can be called again later with different inputs.

In [ ]:
def create_simulation(
    system: openmm.System,
    topology: openmm.app.Topology,
    positions: openmm.unit.Quantity,
    pdb_stride: int = 500,
) -> openmm.app.Simulation:

    barostat = openmm.MonteCarloBarostat(
        1.00 * openmm.unit.bar, 293.15 * openmm.unit.kelvin, 25
    )
    system.addForce(barostat)

    integrator = openmm.LangevinIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        1 * openmm.unit.femtoseconds,
    )

    simulation = openmm.app.Simulation(topology, system, integrator)
    simulation.context.setPositions(positions)

    # https://github.com/openmm/openmm/issues/3736#issuecomment-1217250635
    simulation.minimizeEnergy()

    simulation.context.setVelocitiesToTemperature(300 * openmm.unit.kelvin)
    simulation.context.computeVirtualSites()

    pdb_reporter = openmm.app.PDBReporter("trajectory.pdb", pdb_stride)
    state_data_reporter = openmm.app.StateDataReporter(
        "data.csv", 10, step=True, potentialEnergy=True, temperature=True, density=True
    )
    simulation.reporters.append(pdb_reporter)
    simulation.reporters.append(state_data_reporter)

    return simulation

In [ ]:
simulation = create_simulation(
    system, interchange.topology.to_openmm(), to_openmm(interchange.positions)
)

Finally, we can run this simulation. This should take approximately 10-20 seconds on a laptop or small workstation.

Again, let's wrap this up into a function to avoid copy-pasting code.

In [ ]:
def run_simulation(simulation: openmm.app.Simulation, n_steps: int = 500):

    print("Starting simulation")
    start_time = time.process_time()

    print("Step, box lengths (nm)")

    for step in range(n_steps):
        simulation.step(1)
        if step % 500 == 0:
            box_vectors = simulation.context.getState().getPeriodicBoxVectors()
            print(step, [round(box_vectors[dim][dim]._value, 3) for dim in range(3)])

    end_time = time.process_time()
    print(f"Elapsed time: {(end_time - start_time):.2f} seconds")

In [ ]:
run_simulation(simulation, n_steps=5000)

## Appendix A: visualizing the trajectory

If [NGLView](http://nglviewer.org/nglview/latest/) is installed, we can use it and MDTraj to load and visualize the PDB trajectory:

In [ ]:
# NBVAL_SKIP
try:
    import nglview

    view = nglview.show_mdtraj(mdtraj.load("trajectory.pdb"))
    view
except ImportError:
    pass

## Appendix B: using the TIP4P water model

In [ ]:
from openff.interchange.tests import get_test_file_path

force_field = ForceField(
    "openff_unconstrained-2.0.0.offxml", get_test_file_path("tip4p.offxml")
)

In [ ]:
interchange: Interchange = Interchange.from_smirnoff(
    force_field=force_field, topology=topology
)
system: openmm.System = interchange.to_openmm(combine_nonbonded_forces=True)

interchange.positions = unit.Quantity(
    mdtraj.load("solvated.pdb").xyz[0],
    unit.nanometer,
)

In [ ]:
from openff.interchange.interop.openmm import to_openmm_positions

simulation = create_simulation(
    system,
    interchange.to_openmm_topology(),
    to_openmm(to_openmm_positions(interchange)),
    pdb_stride=100,
)

run_simulation(simulation, n_steps=5000)

In [ ]:
# NBVAL_SKIP
import mdtraj
import nglview

# NGLview likes to infer bonds between virtual sites in ways that look messy,
# but you can flip this to `True` just to ensure they're there
show_virtual_sites = False

if show_virtual_sites:
    view = nglview.show_file("trajectory.pdb")
else:
    import numpy

    trajectory = mdtraj.load("trajectory.pdb")

    atom_indices = numpy.where(
        [a.element.mass > 0.0 for a in trajectory[0].topology.atoms]
    )[0]

    view = nglview.show_mdtraj(trajectory.atom_slice(atom_indices))

view